In [ ]:
## data generators
from tqdm.notebook import tqdm
import numpy as np
def mv_generator(name,shape):
    K = []
    with open(name,'rt') as stream:
        n = 0
        size = shape[0]*shape[1]*2
        for line in stream:
            L = line.split(',')
            K+=[float(L[0].strip()),float(L[1].strip())]
            n+=1
            if n==shape[0]*shape[1]:
                n=0
                r = np.array(K).reshape(shape[0],shape[1],2)
                K = []
                yield r
                
def getID(x):
    ID = int(x.split('_')[-1].split('.')[0])
    return ID

def im_generator(names):
    for im in names:
        yield getID(im), np.array(Image.open(im))
        
# plots
import cv2
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image

select = 'seq1'
images = glob(f'./ssd1/{select}/*.jpg')
im = np.array(Image.open(images[0]))
imshape = im.shape[:-1]
del im
print(len(images),'images found,',imshape)

def fmod(x,y):
    return x-np.floor(x/y)*y

def Sobel3(I):
    R = I.astype('float32')
    Gx = R[:-2,:-2,:]-R[2:,:-2,:]+2*(R[:-2,1:-1,:]-R[2:,1:-1,:])+R[:-2,2:,:]-R[2:,2:,:]
    Gy = R[:-2,:-2,:]-R[:-2,2:,:]+2*(R[1:-1,:-2,:]-R[1:-1,2:,:])+R[2:,:-2,:]-R[2:,2:,:]
    R = np.sqrt(np.sum(Gx**2,axis=2)+np.sum(Gy**2,axis=2))
    return np.pad(R/np.max(R),((1,1),(1,1)),'symmetric')

show = False

for (n,img),mv in tqdm(
    zip(im_generator(images),
        mv_generator(f'./ssd1/mv_{select}/mv_{select[:-1]}_{select[-1]}.txt',imshape)
       ),total=len(images)-1
    ):
    # Use Hue, Saturation, Value colour model 
    hsv = np.zeros((mv.shape[0],mv.shape[1],3), dtype='float32')
    hsv[... ,1] = 0.5
    mag = np.sqrt(mv[..., 0]**2+mv[..., 1]**2)
    ang = np.arctan2(mv[...,1],mv[...,0])
    hsv[..., 0] = fmod(ang / (2*np.pi) * 255,255)
    hsv[..., 2] = 0+1*mag/np.max(mag)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    Srgb = Sobel3(mv)
    Simg = Sobel3(img)
    
    if show:
        fig, ax = plt.subplots(1,1,figsize=(12.8,7.2))
        ax.imshow(img)
        ax.axis('off')
        plt.show()
    out = Image.fromarray(img.astype('uint8'))
    if out.mode!='RGB': out = out.convert('RGB')
    out.save(f'./ssd1/o{select}/{n:04d}rgb.png')
    
    if show:
        fig, ax = plt.subplots(1,1,figsize=(12.8,7.2))
        ax.imshow(1-Simg,cmap='gray')
        ax.axis('off')
        plt.show()
    out = Image.fromarray((255*(1-Simg)).astype('uint8'))
    if out.mode!='L': out = out.convert('L')
    out.save(f'./ssd1/o{select}/{n:04d}rgb_edge.png')
    
    if show:
        fig, ax = plt.subplots(1,1,figsize=(12.8,7.2))
        ax.imshow(rgb)
        ax.axis('off')
        plt.show()
    out = Image.fromarray((255*rgb).astype('uint8'))
    if out.mode!='RGB': out = out.convert('RGB')
    out.save(f'./ssd1/o{select}/{n:04d}mv.png')
    
    if show:
        fig, ax = plt.subplots(1,1,figsize=(12.8,7.2))
        ax.imshow(1-Srgb,cmap='gray')
        ax.axis('off')
        plt.show()
    out = Image.fromarray((255*(1-Srgb)).astype('uint8'))
    if out.mode!='L': out = out.convert('L')
    out.save(f'./ssd1/o{select}/{n:04d}mv_edge.png')